In [33]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import joblib
from flask import Flask, render_template, request, send_from_directory, jsonify, send_file, redirect
import requests

df = pd.read_csv('Pokemon.csv')

df



dflegend = df[['Legendary']].replace(False, 'Not Legend')
dflegend = dflegend[['Legendary']].replace(True, 'Legend')
df = df.drop(['Legendary'], axis=1)
df['Legendary'] = dflegend
df
# df

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,Not Legend
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,Not Legend
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,Not Legend
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,Not Legend
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,Not Legend
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,Not Legend
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,Not Legend
7,6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,Not Legend
8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,Not Legend
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43,1,Not Legend


In [34]:
# dfimg = df[['no','digimon','image']]
# dfimg['digimon'] = dfimg['digimon'].apply(lambda x:x.capitalize())
dfre = df[['#','Name','Type 1','Generation', 'Legendary']]
def mergecol(i):
    return str(i['Type 1'])+'apalu'+str(i['Generation'])+'apalu'+str(i['Legendary'])
dfre['feature'] = dfre.apply(mergecol,axis='columns')


dfre





//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,#,Name,Type 1,Generation,Legendary,feature
0,1,Bulbasaur,Grass,1,Not Legend,Grassapalu1apaluNot Legend
1,2,Ivysaur,Grass,1,Not Legend,Grassapalu1apaluNot Legend
2,3,Venusaur,Grass,1,Not Legend,Grassapalu1apaluNot Legend
3,3,VenusaurMega Venusaur,Grass,1,Not Legend,Grassapalu1apaluNot Legend
4,4,Charmander,Fire,1,Not Legend,Fireapalu1apaluNot Legend
5,5,Charmeleon,Fire,1,Not Legend,Fireapalu1apaluNot Legend
6,6,Charizard,Fire,1,Not Legend,Fireapalu1apaluNot Legend
7,6,CharizardMega Charizard X,Fire,1,Not Legend,Fireapalu1apaluNot Legend
8,6,CharizardMega Charizard Y,Fire,1,Not Legend,Fireapalu1apaluNot Legend
9,7,Squirtle,Water,1,Not Legend,Waterapalu1apaluNot Legend


In [35]:
dfre['Name'] = dfre['Name'].apply(lambda x:x.capitalize())
dfre.to_string()
model = CountVectorizer(
    tokenizer = lambda i: i.split('apalu'),
    analyzer = 'word',
)
matrix = model.fit_transform(dfre['feature'])
matrix
model.get_feature_names()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 'bug',
 'dark',
 'dragon',
 'electric',
 'fairy',
 'fighting',
 'fire',
 'flying',
 'ghost',
 'grass',
 'ground',
 'ice',
 'legend',
 'normal',
 'not legend',
 'poison',
 'psychic',
 'rock',
 'steel',
 'water']

In [36]:
dfmx = matrix.toarray()
score = cosine_similarity(dfmx)
score

array([[1.        , 1.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 1.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 1.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        0.66666667],
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        0.66666667],
       [0.        , 0.        , 0.        , ..., 0.66666667, 0.66666667,
        1.        ]])

In [37]:
# pokename = request.form['pokemon'].capitalize()
pokename = input('name: ')
nomor = df[df['Name'] == pokename.capitalize()]['#'].tolist()[0]
type(nomor)
# if pokename not in list(dfre['Name']):
#     return redirect ('/notfound')

    # get the pokemon that what we want
# imgre = dfimg['image'][dfimg['digimon']==diginame.capitalize()].values[0]
typepoke = dfre['Type 1'][dfre['Name']==pokename.capitalize()].values[0]
genpoke = dfre['Generation'][dfre['Name']==pokename.capitalize()].values[0]
legpoke = dfre['Legendary'][dfre['Name']==pokename.capitalize()].values[0]

legpoke

name: pikachu


'Not Legend'

In [38]:
likeIndex = dfre['#'][dfre['Name'] == pokename.capitalize()].values[0]
allpoke = list(enumerate(score[likeIndex]))
for i in range(len(allpoke)):
    if int(allpoke[i][0]) != int(likeIndex):
        similarpoke = (sorted(allpoke,key = lambda x: x[1],reverse = True))
    
similarpoke
# likeIndex
# df[dfre['#'] == listrecom[i][0]]
# dfre['#']
# len(listrecom)



[(20, 1.0000000000000002),
 (21, 1.0000000000000002),
 (22, 1.0000000000000002),
 (23, 1.0000000000000002),
 (24, 1.0000000000000002),
 (25, 1.0000000000000002),
 (26, 1.0000000000000002),
 (27, 1.0000000000000002),
 (44, 1.0000000000000002),
 (45, 1.0000000000000002),
 (57, 1.0000000000000002),
 (58, 1.0000000000000002),
 (90, 1.0000000000000002),
 (91, 1.0000000000000002),
 (92, 1.0000000000000002),
 (116, 1.0000000000000002),
 (121, 1.0000000000000002),
 (123, 1.0000000000000002),
 (124, 1.0000000000000002),
 (138, 1.0000000000000002),
 (143, 1.0000000000000002),
 (144, 1.0000000000000002),
 (148, 1.0000000000000002),
 (155, 1.0000000000000002),
 (0, 0.6666666666666669),
 (1, 0.6666666666666669),
 (2, 0.6666666666666669),
 (3, 0.6666666666666669),
 (4, 0.6666666666666669),
 (5, 0.6666666666666669),
 (6, 0.6666666666666669),
 (7, 0.6666666666666669),
 (8, 0.6666666666666669),
 (9, 0.6666666666666669),
 (10, 0.6666666666666669),
 (11, 0.6666666666666669),
 (12, 0.6666666666666669),
 (

In [39]:
listrecom = []
for i in range(len(similarpoke)):
    if int(similarpoke[i][0]) != int(likeIndex):
        listrecom.append(similarpoke[i])
newlist = []
typeoth = []
genoth = []
legoth = []
for i in range(0,len(listrecom[:6])):
    newlist.append(dfre['Name'][dfre['#']==listrecom[i][0]].values[0])
    typeoth.append(dfre['Type 1'][dfre['#']==listrecom[i][0]].values[0])
    genoth.append(dfre['Generation'][dfre['#']==listrecom[i][0]].values[0])
    legoth.append(dfre['Legendary'][dfre['#']==listrecom[i][0]].values[0])
legoth


# listrecom = []
# for i in range(len(similardigi)):
#     if int(similardigi[i][0]) != int(likeIndex):
#         listrecom.append(similardigi[i])
# newlist = []
# imgsrc = []
# stgoth = []
# typeoth = []
# attoth = []
# for i in range(len(listrecom[:6])):
#     newlist.append(dfre['digimon'][dfre['no']==listrecom[i][0]].values[0])
#     imgsrc.append(dfimg['image'][dfimg['no']==listrecom[i][0]].values[0])
#     stgoth.append(dfre['stage'][dfre['no']==listrecom[i][0]].values[0])
#     typeoth.append(dfre['type'][dfre['no']==listrecom[i][0]].values[0])
#     attoth.append(dfre['attribute'][dfre['no']==listrecom[i][0]].values[0])



['Not Legend',
 'Not Legend',
 'Not Legend',
 'Not Legend',
 'Not Legend',
 'Not Legend']

In [40]:
# listrecom = []
# for i in range(len(similarpoke)):
#     if int(similarpoke[i][0]) != int(likeIndex):
#         listrecom.append(similarpoke[i])
# newlist = []
# # imgsrc = []
# typeoth = []
# genoth = []
# legoth = []
# for i in range(len(listrecom[:6])):
#     newlist.append(dfre['Name'][dfre['#']==listrecom[i][0]].values[0])
# #     imgsrc.append(dfimg['image'][dfimg['no']==listrecom[i][0]].values[0])
#     typeoth.append(dfre['Type 1'][dfre['#']==listrecom[i][0]].values[0])
#     genoth.append(dfre['Generation'][dfre['#']==listrecom[i][0]].values[0])
#     legoth.append(dfre['Legendary'][dfre['#']==listrecom[i][0]].values[0])

In [32]:
url1 = f'https://pokeapi.co/api/v2/pokemon/{pokename.lower()}'
# url1 = f'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/shiny/{nomor}.png'
data1 = requests.get(url1)

# data1.json()['sprites']['front_default']
data1.json()['forms']


# https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/shiny/25.png

KeyError: 'forms'

In [10]:
# https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/back/25.png  ## pikachu